**<프로토타입용 추천 시스템>**

- 하이브리드 기반 추천 시스템(Hybrid Recommender Systems)

(사용한) 머신러닝을 위한 파이썬 라이브러리
- Numpy(넘파이)
- Pandas(판다스)
- sklearn(사이킷런)
- surprise(서프라이즈)

* 두 가지 필터링 방법을 결합하여 하나의 추천 시스템을 개발한다.
  - 해당 유저와 비슷한 유저가 구매/조회한 아이템이면서, 그 아이템의 다른 추가 속성(ex. 유사한 장르/유사한 카테고리 등)에 있는 다른 아이템도 추가로 추천하는 방식

- [하이브리드 기반 추천 시스템]
  - 콘텐츠 기반 필터링과 협업 기반 필터링을 합친 추천 시스템
  - CF(Collaborative Filtiering) 점수에 콘텐츠 기반의 점수를 곱해서 정렬 후 사용

  - 하이브리드 모델 방식: 콘텐츠 기반 필터링에서 시대를 이용하여 유사한 명화를 추천해주는 방식과 협업 기반 필터링에서 평점 데이터를 이용해 개인이 평점을 남긴 평점 데이터를 기반으로 새롭게 모든 명화의 평점을 예측하여 높은 예측 평점 중 개인이 평점을 부여하지 않은 명화만을 추천하는 방식을 결합하여 만든다.

----------

- 필요한 라이브러리 import 

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [ ]:
from surprise import SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import dump

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

- 데이터 준비 및 전처리



In [ ]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(0, 5))
    trainset = Dataset.load_from_df(training_dataframe[['user_id', 'masterpiece_seq', 'grade_score']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['user_id', 'masterpiece_seq', 'grade_score']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    return trainset, testset

In [ ]:
masterpiece = pd.read_csv('https://raw.githubusercontent.com/usernamexuxi/WOOYEWEB/main/RecommendationSystem/csv/masterpiece.csv?token=GHSAT0AAAAAAB2RKSYSXJKCTAB3CCATACDGY25JXSA', encoding='utf-8')
grade = pd.read_csv('https://raw.githubusercontent.com/usernamexuxi/WOOYEWEB/main/RecommendationSystem/csv/grade.csv?token=GHSAT0AAAAAAB2RKSYS2ZVQS55UVPZFYQ2WY25JX7A', encoding='utf-8')

masterpiece_grade = pd.merge(masterpiece, grade, on="masterpiece_seq")
masterpiece_grade.drop(['masterpiece_picture', 'masterpiece_footnote'], axis=1, inplace=True)

traindf = masterpiece_grade
testdf = masterpiece_grade
trainset, testset = convert_traintest_dataframe_forsurprise(traindf, testdf)

In [ ]:
testdf.head()

,masterpiece_seq,masterpiece_title,masterpiece_artist,masterpiece_era,masterpiece_year,masterpiece_technique,masterpiece_material,grade_seq,user_id,grade_score
0,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,2,이소이,8
1,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,44,이루다,1
2,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,92,지사해,8
3,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,135,탁재영,4
4,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,209,김수연,6


- 협업 필터링(CF, Collaborative Filtering) KNN(K Nearest Neighbors) 모델
  - K명의 최근접 이웃에 기반해서 찾는 방법
  - 사용자가 준 평점으로 유사한 사람의 아이템을 찾거나, 유사한 아이템을 찾아 추천함
  - 편향을 제거(전반적으로 평점을 후하게 주거나 적게 주는 경우를 방지)해주기 위해 비교군의 평점을 더해주거나 빼주어 동일하게 해준다.

In [ ]:
# basic collaborative filtering algorithm taking into account a baseline rating.
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
knnbaseline_algo = KNNBaseline(sim_options=sim_options)

knnbaseline_algo.fit(trainset)
knnbaseline_predictions = knnbaseline_algo.test(testset)

file_name = 'KnnBaseline_model'
dump.dump(file_name, algo=knnbaseline_predictions)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(knnbaseline_predictions)
accuracy.mae(knnbaseline_predictions)
print("Done!")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 2.8936
MAE:  2.4975
Done!


- 협업 필터링(CF, Collaborative Filtering) SVD(Singular Value Decomposition) 모델
  - 고유값 분해의 일반화 버전
  - 여기서 말하는 일반화란, 고유값 분해가 m×m 정사각 대칭행렬에서 성립되었다면 특이값 분해는 반드시 정사각행렬일 필요도, 대칭행렬일 필요가 없는 것. 
  - 임의의 행렬 n×p에 대해서도 성립하는 것이 특이값 분해. (즉, 제약이 없어진다는 뜻)
  > 특이값 분해를 이용하면 어떤 종류의 행렬이라도 분해 가능
  - 행렬을 차원축소 하기위한 도구로, 실제로는 주성분 분석(PCA, principle component analysis)과 같은 차원축소 분야에서는 특이값 분해가 흔하게 쓰임

In [ ]:
svdpp_algo = SVDpp() # SVDpp 알고리즘은 SVD의 연장

svdpp_algo.fit(trainset)
svdpp_predictions = svdpp_algo.test(testset)

file_name = 'svd_model'
dump.dump(file_name, algo=svdpp_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svdpp_predictions)
accuracy.mae(svdpp_predictions)
print("Done!")

RMSE: 2.5058
MAE:  2.0099
Done!


- Masterpiece Similarity model(명화 유사도 모델)
  - 유사도 함수는 자신과 가장 비슷한 컨텐츠를 찾아줄 때 사용

- 코사인 유사도
  - 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미
  - 두 벡터의 방향이 완전히 동일한 경우에는 1의 값을 가지며, 90°의 각을 이루면 0, 180°로 반대의 방향을 가지면 -1의 값을 갖게 됨
  - 즉, 코사인 유사도는 -1 이상 1 이하의 값을 가지며 값이 1에 가까울수록 유사도가 높다고 판단할 수 있음.
> "두 벡터가 가리키는 방향이 얼마나 유사한가를 의미"

In [ ]:
era_to_idx = {
    '르네상스' : 1,
    '바로크' : 2,
    '로코코' : 3,
    '고전주의' : 4,
    '낭만주의' : 5,
    '사실주의' : 6,
    '인상주의' : 7,
    '표현주의' : 8,
    '초현실주의' : 9,
    '팝아트' : 10,
}

In [ ]:
idx_to_era = {
    1 : '르네상스',
    2 : '바로크',
    3 : '로코코',
    4 : '고전주의',
    5 : '낭만주의',
    6 : '사실주의',
    7 : '인상주의',
    8 : '표현주의',
    9 : '초현실주의',
    10 : '팝아트',
}

In [ ]:
masterpieces = masterpiece[['masterpiece_seq', 'masterpiece_title', 'masterpiece_artist', 'masterpiece_era', 'masterpiece_year', 'masterpiece_technique', 'masterpiece_material']]

# masterpieces['description_era'] = masterpieces['masterpiece_technique'] + 2*masterpieces['masterpiece_era']
masterpieces['description_era'] = masterpieces['masterpiece_era']
masterpieces['description_era'] = masterpieces['description_era'].fillna('')

In [ ]:
tf_new = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0)
tfidf_matrix_new = tf_new.fit_transform(masterpieces['description_era'])

In [ ]:
cosine_sim_new = linear_kernel(tfidf_matrix_new, tfidf_matrix_new) # 코사인 유사도를 구함

In [ ]:
masterpieces = masterpieces.reset_index()
titles = masterpieces['masterpiece_title']
indices = pd.Series(masterpieces.index, index=masterpieces['masterpiece_title'])
indices.head(2)

masterpiece_title
모나리자    0
천지창조    1
dtype: int64

In [ ]:
# get_recommendations_new 함수로 'masterpiece_title'를 보내면, 해당 명화의 유사한 시대의 'masterpiece_seq' 반환

def get_recommendations_new(title):
    idx = indices[title]
    if type(idx) != np.int64:
        if len(idx)>1:
            print("ALERT: Multiple values")
            idx = idx[0]
    sim_scores = list(enumerate(cosine_sim_new[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    # print(sim_scores)
    masterpiece_indices = [i[0] for i in sim_scores]
    # print(masterpiece_indices)
    return masterpieces['masterpiece_seq'].iloc[masterpiece_indices]

In [ ]:
get_recommendations_new('오감')

6      7
7      8
8      9
9     10
10    11
11    12
12    13
0      1
1      2
2      3
Name: masterpiece_seq, dtype: int64

- Popularity model
  -  era_based_popularity 함수
  - user_top_era 함수



In [ ]:
masterpiece_grade.head(20)

,masterpiece_seq,masterpiece_title,masterpiece_artist,masterpiece_era,masterpiece_year,masterpiece_technique,masterpiece_material,grade_seq,user_id,grade_score
0,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,2,이소이,8
1,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,44,이루다,1
2,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,92,지사해,8
3,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,135,탁재영,4
4,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,209,김수연,6
5,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,223,남리을,7
6,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,342,신아연,7
7,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,385,서희수,4
8,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,394,김성연,2
9,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,444,최수현,1


In [ ]:
# 시대를 넣으면 해당 시대의 명화에 부여된 평점들을 내림차순으로 정렬하고 이 중 상위 10개의 'masterpiece_seq' 출력

def era_based_popularity(era):
    mask = masterpiece_grade.masterpiece_era.apply(lambda x: era in x)
    filtered_masterpiece = masterpiece_grade[mask]
    filtered_masterpiece = filtered_masterpiece.sort_values(by='grade_score', ascending=False)
#     filtered_masterpiece = filtered_masterpiece.sort_values(by='wr', ascending=False)
    return filtered_masterpiece['masterpiece_seq'].head(10).values.tolist()

In [ ]:
era_based_popularity('르네상스')

[2, 3, 5, 4, 2, 1, 3, 3, 5, 4]

In [ ]:
user = pd.read_csv('https://raw.githubusercontent.com/usernamexuxi/WOOYEWEB/main/RecommendationSystem/csv/user.csv?token=GHSAT0AAAAAAB2RKSYT64CIENL7WVUWIZL4Y25JY6A', encoding='utf-8')
user_grade = pd.merge(grade, user, on="user_id")
user_grade.head(3)

,grade_seq,user_id,masterpiece_seq,grade_score,user_gender,user_age
0,1,이소이,2,6,0,17
1,2,이소이,1,8,0,17
2,3,이소이,14,2,0,17


In [ ]:
# user_top_era 함수로 'user_id'를 보내면, 해당 유저가 평점을 준 명화의 시대 중 빈도가 높은 시대 세 가지 반환

from numpy.ma.core import append
import collections

def user_top_era(user_id):
    bq2 = masterpiece_grade[['user_id', 'masterpiece_seq','masterpiece_era']]

    user_era = bq2['masterpiece_era'][masterpiece_grade['user_id'] == user_id]
    # print("User Era: ", user_era)
    
    top_era_indices = np.flip(user_era)
    counts = collections.Counter(top_era_indices)
    counts_w = sorted(counts.items(), key= lambda item: item[1], reverse=True)
    
    era_dict = dict(counts_w)

    era_list = list(era_dict.keys())

    return era_list[:3]

In [ ]:
user_top_era('이소이')

['바로크', '팝아트', '르네상스']

- Hybrid model

In [ ]:
knn_baseline = dump.load('KnnBaseline_model')
svd = dump.load('svd_model') 

In [ ]:
# List of users in testing data:
user_list = testdf['user_id'].unique()

In [ ]:
# type(testdf['userId'][0])
test_masterpiece = testdf[testdf['user_id'] == '이소이']
test_masterpiece.head()

,masterpiece_seq,masterpiece_title,masterpiece_artist,masterpiece_era,masterpiece_year,masterpiece_technique,masterpiece_material,grade_seq,user_id,grade_score
0,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,2,이소이,8
14,2,천지창조,미켈란젤로 부오나로티,르네상스,1512.0,종교화,벽화,1,이소이,6
37,4,아테네 학당,라파엘로 산치오,르네상스,1510.0,추상화,벽화,25,이소이,1
52,5,최후의 심판,미켈란젤로 부오나로티,르네상스,1541.0,종교화,벽화,26,이소이,5
65,6,십자가에서 내려지는 그리스도,피터 반 몰,바로크,1634.0,종교화,유화,9,이소이,7


In [ ]:
# Combined model predicion on testing data, using top masterpieces to era more masterpieces based on masterpiece similarity
# 테스트 데이터에 대한 결합 모델 예측 / 유사도를 기반으로, 동시대이며 높은 평점를 지닌 명화를 사용하여 더 많은 유사 명화를 추천

def hybrid(user_id):
    user_masterpiece = testdf[testdf['user_id'] == user_id]
    user_masterpiece['est'] = user_masterpiece['masterpiece_seq'].apply(lambda x: 0.6*knnbaseline_algo.predict(user_id,x).est + 0.4*svdpp_algo.predict(user_id, x).est)    
    user_masterpiece = user_masterpiece.sort_values(by ='est', ascending=False).head(4)
    user_masterpiece['Model'] = 'SVD + CF'
#     user_masterpiece = user_masterpiece['masterpiece_seq'].values.tolist()
#     print("User liked masterpiece list: ", user_masterpiece)
    
    recommend_list = user_masterpiece[['masterpiece_seq', 'est', 'Model']]
    print(recommend_list.head())

#     top_masterpiece = user_masterpiece['masterpiece_seq'].iloc[0]
#     print("Top masterpiece id", top_masterpiece)
#     top_masterpiece_title = masterpiece['masterpiece_title'][masterpiece['masterpiece_seq'] == top_masterpiece].values[0]
#     print("Top masterpiece title", top_masterpiece_title)

    
    masterpiece_list = recommend_list['masterpiece_seq'].values.tolist()
    print(masterpiece_list)
    
    sim_masterpiece_list = []
    for masterpieces_id in masterpiece_list:
        # Call content based 
        # 콘텐츠 기반 호출
        masterpiece_title = masterpieces['masterpiece_title'][masterpieces['masterpiece_seq'] == masterpieces_id].values[0]
        sim_masterpiece = get_recommendations_new(masterpiece_title)
#       print(sim_masterpiece.values.tolist())
        sim_masterpiece_list.extend(sim_masterpiece)

    for masterpieces_id in sim_masterpiece_list:
        pred_rating = 0.6*knnbaseline_algo.predict(user_id, masterpieces_id).est + 0.4*svdpp_algo.predict(user_id, masterpieces_id).est
        row_df = pd.DataFrame([[masterpieces_id, pred_rating, 'Movie similarity']], columns=['masterpiece_seq', 'est','Model'])
        recommend_list = pd.concat([recommend_list, row_df], ignore_index=True)
    
    # Popular based masterpieces
    # 유사도가 높은 명화
    top_era_list = user_top_era(user_id)
    print("User top era list: ", top_era_list)
    
    popular_masterpiece = []
    for top_era in top_era_list:
        popular_masterpiece.extend(era_based_popularity(top_era))
    print("Final list: ", popular_masterpiece)
    
    # Compute ratings for the popular masterpieces
    # 유사도가 높은 명화에 대한 평점 계산

    for masterpieces_id in popular_masterpiece:
        pred_rating = 0.6*knnbaseline_algo.predict(user_id, masterpieces_id).est + 0.4*svdpp_algo.predict(user_id, masterpieces_id).est
        row_df = pd.DataFrame([[masterpieces_id, pred_rating, 'Popularity']], columns=['masterpiece_seq', 'est','Model'])
        recommend_list = pd.concat([recommend_list, row_df], ignore_index=True)
    recommend_list = recommend_list.drop_duplicates(subset=['masterpiece_seq'])
    train_masterpieces_list = traindf[traindf['user_id']==user_id]['masterpiece_seq'].values.tolist()
    
    # Remove masterpieces in training for this user
    # 해당 유저가 기존 데이터 중 평점을 부여한 적 있는 명화를 제외 
    mask = recommend_list.masterpiece_seq.apply(lambda x: x not in train_masterpieces_list)
    recommend_list = recommend_list[mask]
    
    return recommend_list

In [ ]:
# traindf[traindf['user_id'] == '이소이'].sort_values(by = 'grade_score', ascending = False)
traindf[traindf['user_id'] == '이소이'].sort_values(by = 'grade_score', ascending = False)
# testdf[testdf['user_id'] == '이소이']

,masterpiece_seq,masterpiece_title,masterpiece_artist,masterpiece_era,masterpiece_year,masterpiece_technique,masterpiece_material,grade_seq,user_id,grade_score
569,51,캠벨 수프 통조림,앤디 워홀,팝아트,1962.0,정물화,실크 스크린,19,이소이,10
294,25,파가니니의 초상,외젠 들라크루아,낭만주의,1832.0,인물화,유화,12,이소이,10
486,43,나와 마을,마르크 샤갈,표현주의,1911.0,추상화,유화,17,이소이,10
444,39,절규,에드바르 뭉크,표현주의,1893.0,추상화,유화,24,이소이,10
103,9,정물,헤다,바로크,1634.0,정물화,유화,20,이소이,9
0,1,모나리자,레오나르도 다 빈치,르네상스,1503.0,인물화,유화,2,이소이,8
559,50,빛나는 아기,키스 해링,팝아트,1990.0,추상화,벽화,6,이소이,8
194,16,제르생의 간판,장 앙투안 와토,로코코,1720.0,인물화,유화,10,이소이,8
113,10,와인잔이 있는 정물,피터 클레즈,바로크,1642.0,정물화,유화,27,이소이,8
320,27,발코니 방,아돌프 멘젤,낭만주의,1845.0,정물화,유화,7,이소이,8


In [ ]:
masterpiece_ids = hybrid('장건우')

     masterpiece_seq  est     Model
206               16  5.0  SVD + CF
493               43  5.0  SVD + CF
400               34  5.0  SVD + CF
141               11  5.0  SVD + CF
[16, 43, 34, 11]
User top era list:  ['바로크', '로코코', '초현실주의']
Final list:  [8, 6, 12, 13, 13, 10, 6, 11, 8, 9, 16, 18, 18, 17, 17, 16, 18, 18, 18, 16, 48, 47, 47, 45, 45, 44, 46, 48, 47, 47]


In [ ]:
def get_title(x):
    mid = x['masterpiece_seq']
    return masterpiece['masterpiece_title'][masterpiece['masterpiece_seq'] == mid].values

In [ ]:
def get_era(x):
    mid = x['masterpiece_seq']
    return masterpiece['masterpiece_era'][masterpiece['masterpiece_seq'] == mid].values

In [ ]:
masterpiece_ids['masterpiece_title'] = masterpiece_ids.apply(get_title, axis=1)
masterpiece_ids['masterpiece_era'] = masterpiece_ids.apply(get_era, axis=1)

In [ ]:
# 'user_id'가 아직 평점 내리지 않은 명화 중 est가 높은 순으로 추천
masterpiece_ids.sort_values(by='est', ascending = False).head(10)

,masterpiece_seq,est,Model,masterpiece_title,masterpiece_era
7,18,5.000000,Movie similarity,[미델하르니스의 가로수길],[로코코]
12,5,5.000000,Movie similarity,[최후의 심판],[르네상스]
13,6,5.000000,Movie similarity,[십자가에서 내려지는 그리스도],[바로크]
26,37,4.895376,Movie similarity,[사과 바구니],[인상주의]
8,1,4.879758,Movie similarity,[모나리자],[르네상스]
67,45,4.832333,Popularity,[연인들],[초현실주의]
14,40,4.762854,Movie similarity,[더 키스],[표현주의]
70,46,4.616315,Popularity,[기억의 지속],[초현실주의]
25,36,4.425275,Movie similarity,[해바라기],[인상주의]
15,41,4.251325,Movie similarity,[꽃피는 아몬드 나무],[표현주의]
